In [1]:
import scapy
import subprocess
from scapy.all import rdpcap, sniff, get_if_list, conf, Scapy_Exception, get_if_hwaddr
from collections import Counter
import pandas as pd
import socket
import os
import time
import netifaces

In [2]:
def get_packets(interface = "ethernet_0", filename = "sniff_cap.csv", count = 100, print_status = False, timeout=None):
    """
    This function is designed to capture a number of ip packets to analyize with pandas.
    """
    protocol_numbers = {1: "ICMP", 2: "IGMP", 6: "TCP", 17: "UDP", 41: "IPv6",
                        47: "GRE", 50: "ESP", 51: "AH", 58: "ICMPv6", 89: "OSPF",
                        103: "PIM", 132: "SCTP"}

    #Checks if file exists
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        return df
    #Sniffs on all interfaces for 'count' number of ip packets
    else:    
        sniffed_packets = sniff(iface=interface, count=count, filter="ip", timeout=timeout)
        list_of_packets = []
        i = 0

        for packet in sniffed_packets:
            if packet.haslayer('IP'):
                try:
                    source_name = socket.gethostbyaddr(packet['IP'].src)
                except socket.herror:
                    source_name = None
                try:
                    destination_name = socket.gethostbyaddr(packet['IP'].dst)
                except socket.herror:
                    destination_name = None
                i += 1
                #Keeps track of how many captures and prints to screen
                if print_status == True:
                    if i % (count/10) == 0:
                        print(f"Percent complete: {(i/count) * 100} %")

                #Collects the information into a dictionary for later use in Pandas
                protocol_number = packet['IP'].proto
                
                temp_dict = {"protocol_name": protocol_numbers.get(protocol_number, "Unknown"),
                             "src_IP" : str(packet["IP"].src),
                             "src_name": source_name,
                             "dst_IP" : str(packet["IP"].dst),
                             "dst_name": destination_name,
                             "payload": str(packet["IP"].payload)}
                if packet.haslayer('TCP'):
                    temp_dict["source_port"] = str(packet["TCP"].sport)
                    temp_dict["dst_port"] = str(packet["TCP"].dport)
                    
                elif packet.haslayer('UDP'):
                    temp_dict["source_port"] = str(packet["UDP"].sport)
                    temp_dict["dst_port"] = str(packet["UDP"].dport)

                else:
                    temp_dict["source_port"] = None
                    temp_dict["dst_port"] = None
                list_of_packets.append(temp_dict)
                
        df = pd.DataFrame(list_of_packets)
        df.to_csv(filename, index=False)

        return df

In [3]:
list_of_ifaces = get_if_list()

In [4]:
#print(list_of_ifaces)

In [5]:
# Get a list of network interfaces
interfaces = netifaces.interfaces()

# Print details for each interface
for interface in interfaces:
    #print(f'Interface: {interface}')
    
    # Get the addresses for this interface
    addresses = netifaces.ifaddresses(interface)
    """
    # Print the MAC address, if available
    if netifaces.AF_LINK in addresses:
        print('MAC Address:', addresses[netifaces.AF_LINK][0]['addr'])
    
    # Print the IPv4 address, if available
    if netifaces.AF_INET in addresses:
        print('IPv4 Address:', addresses[netifaces.AF_INET][0]['addr'])
    
    # Print the IPv6 address, if available
    if netifaces.AF_INET6 in addresses:
        print('IPv6 Address:', addresses[netifaces.AF_INET6][0]['addr'])
    """
    #print()


In [6]:
# Specify the name of the .pcap file
pcap_file = 'capture.pcap'

# Specify the Wi-Fi interface
interface = interfaces[9]

# Specify the capture duration in seconds
capture_duration = 60

# Run Tshark to capture packets
tshark_command = ["C:\\Program Files\\Wireshark\\tshark.exe", "-i", interface, "-a", f"duration:{capture_duration}", "-w", pcap_file]

# Create the .pcap file if it does not exist yet
if not os.path.exists(pcap_file):
   subprocess.run(tshark_command)

In [7]:
try:
    # Use Scapy to read the .pcap file
    packets = rdpcap(pcap_file)

except FileNotFoundError:
    print(f"The file{pcap_file} does not exist yet.")

    # Analyze the packets
ip_packets = [pkt for pkt in packets if pkt.haslayer('IP')]
tcp_packets = [pkt for pkt in packets if pkt.haslayer('TCP')]
udp_packets = [pkt for pkt in packets if pkt.haslayer('UDP')]

print(f'Captured {len(packets)} packets total:')
print(f'{len(ip_packets)} IP packets')
print(f'{len(tcp_packets)} TCP packets')
print(f'{len(udp_packets)} UDP packets')

The filecapture.pcap does not exist yet.


NameError: name 'packets' is not defined